# Artificial Intelligence: NLP Techniques on Mental Health Subreddit
(In progress)

In [1]:
import pandas as pd
import praw  
import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from wordcloud import WordCloud, ImageColorGenerator 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
headers = {'User-Agent': 'text_analytics'}

## DATA COLLECTION: REDDIT SCRAPE

In [2]:
reddit = praw.Reddit(client_id=' ', 
                     client_secret=' ', user_agent='username')

subreddit = reddit.subreddit('mentalhealth')

In [3]:
df = pd.DataFrame()
for submission in subreddit.new(limit=5):
    df = df.append({'Title':submission.title,
                  'Content':submission.selftext,
                  'ups':submission.ups,
                  'user':submission.author,
                  'url':submission.url,
                  'downs':submission.downs},ignore_index=True)

df

,Title,Content,ups,user,url,downs
0,An estranged friend tried to call me. TW: suic...,"I'm feeling very weird and confused right now,...",1.0,secrethomoshh,https://www.reddit.com/r/mentalhealth/comments...,0.0
1,“Adult” Only Children,"Firstly, I love my family very much. I am very...",2.0,reditb2021,https://www.reddit.com/r/mentalhealth/comments...,0.0
2,"What should I do if I need a therapist, but ca...","I think I need more answers, help, and to unde...",1.0,Illiolite,https://www.reddit.com/r/mentalhealth/comments...,0.0
3,Please help,TLDR: I prefer being alone all the time. Even ...,1.0,Less-Relation8428,https://www.reddit.com/r/mentalhealth/comments...,0.0
4,Anxiety attacks,I've been having extreme anxiety attacks every...,1.0,PoohScreech,https://www.reddit.com/r/mentalhealth/comments...,0.0


##  Part-of-Speech Tagging Analysis

In [4]:
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

df['POS tagged'] = df['Content'].apply(token_stop_pos)
df.head()

,Title,Content,ups,user,url,downs,POS tagged
0,An estranged friend tried to call me. TW: suic...,"I'm feeling very weird and confused right now,...",1.0,secrethomoshh,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[('m, v), (feeling, v), (weird, a), (confused,..."
1,“Adult” Only Children,"Firstly, I love my family very much. I am very...",2.0,reditb2021,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(Firstly, r), (,, None), (love, v), (family, ..."
2,"What should I do if I need a therapist, but ca...","I think I need more answers, help, and to unde...",1.0,Illiolite,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(think, v), (need, v), (answers, n), (,, None..."
3,Please help,TLDR: I prefer being alone all the time. Even ...,1.0,Less-Relation8428,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(TLDR, n), (:, None), (prefer, v), (alone, r)..."
4,Anxiety attacks,I've been having extreme anxiety attacks every...,1.0,PoohScreech,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[('ve, v), (extreme, a), (anxiety, n), (attack..."


## Lemmatization: Morphological Analysis

In [5]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

df['Lemma'] = df['POS tagged'].apply(lemmatize)
df.head()

,Title,Content,ups,user,url,downs,POS tagged,Lemma
0,An estranged friend tried to call me. TW: suic...,"I'm feeling very weird and confused right now,...",1.0,secrethomoshh,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[('m, v), (feeling, v), (weird, a), (confused,...","'m feel weird confused right , friend lose c..."
1,“Adult” Only Children,"Firstly, I love my family very much. I am very...",2.0,reditb2021,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(Firstly, r), (,, None), (love, v), (family, ...","Firstly , love family much . bless . adult (..."
2,"What should I do if I need a therapist, but ca...","I think I need more answers, help, and to unde...",1.0,Illiolite,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(think, v), (need, v), (answers, n), (,, None...","think need answer , help , understand 's go ..."
3,Please help,TLDR: I prefer being alone all the time. Even ...,1.0,Less-Relation8428,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[(TLDR, n), (:, None), (prefer, v), (alone, r)...","TLDR : prefer alone time . Even loved one , ..."
4,Anxiety attacks,I've been having extreme anxiety attacks every...,1.0,PoohScreech,https://www.reddit.com/r/mentalhealth/comments...,0.0,"[('ve, v), (extreme, a), (anxiety, n), (attack...",'ve extreme anxiety attack every morning poi...


## Sentiment Analysis

In [6]:
df_data = pd.DataFrame(df[['Content', 'Lemma']])

analyzer = SentimentIntensityAnalyzer()


def polarity(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']

df_data['Polarity'] = df_data['Lemma'].apply(polarity)
    

def sentimentanalysis(compound):
    if compound >= 0.5:
        return 'Positive'
    elif compound <= -0.5 :
        return 'Negative'
    else:
        return 'Neutral'
    
    
df_data['Analysis'] = df_data['Polarity'].apply(sentimentanalysis)
df_data.head()

,Content,Lemma,Polarity,Analysis
0,"I'm feeling very weird and confused right now,...","'m feel weird confused right , friend lose c...",-0.8850,Negative
1,"Firstly, I love my family very much. I am very...","Firstly , love family much . bless . adult (...",0.9415,Positive
2,"I think I need more answers, help, and to unde...","think need answer , help , understand 's go ...",0.8519,Positive
3,TLDR: I prefer being alone all the time. Even ...,"TLDR : prefer alone time . Even loved one , ...",-0.9779,Negative
4,I've been having extreme anxiety attacks every...,'ve extreme anxiety attack every morning poi...,-0.4351,Neutral


In [7]:
sentiment_cnt = df_data['Analysis'].value_counts()
sentiment_cnt

Negative    2
Positive    2
Neutral     1
Name: Analysis, dtype: int64

## Wordlist

In [8]:
text = df_data['Content'].values 
words =' '.join(text).split()
words = [word for word in words if word.isalnum()]
stop_words = set(stopwords.words('english'))

In [9]:
cleaned_words = [w for w in words if not w.lower() in stop_words]
cleaned_words

['feeling',
 'weird',
 'confused',
 'right',
 'friend',
 'lost',
 'contact',
 'months',
 'messaged',
 'today',
 'sent',
 'messages',
 'like',
 'world',
 'fell',
 'away',
 'end',
 'trying',
 'call',
 'sent',
 'confused',
 'messaged',
 'finally',
 'knew',
 'suggesting',
 'told',
 'call',
 'someone',
 'talk',
 'Currently',
 'get',
 'response',
 'hour',
 '40',
 'minutes',
 'tried',
 'calling',
 'worried',
 'done',
 'hope',
 'gone',
 'answered',
 'know',
 'needed',
 'whole',
 'experience',
 'made',
 'upset',
 'also',
 'little',
 'want',
 'able',
 'prevent',
 'could',
 'done',
 'love',
 'family',
 'blessed',
 'adult',
 'child',
 'expected',
 'almost',
 'everything',
 'support',
 'parents',
 'hopes',
 'one',
 'vent',
 'lot',
 'expectations',
 'anyone',
 'tips',
 'setting',
 'boundaries',
 'stepping',
 'would',
 'neverwant',
 'kids',
 'grow',
 'like',
 'think',
 'need',
 'understand',
 'going',
 'kind',
 'want',
 'find',
 'something',
 'way',
 'getting',
 'one',
 'good',
 'alternatives',
 'any

## Vectorization

In [10]:
cv = CountVectorizer(stop_words='english')
cv_matrix = cv.fit_transform(df_data['Content'])

df_dtm = pd.DataFrame(cv_matrix.toarray(),
                      index=df_data['Content'].values,
                      columns=cv.get_feature_names())
df_dtm


,2018,21m,30,3am,40,able,adult,alternatives,answered,answers,anxiety,approve,attacks,away,background,big,biological,blessed,boundaries,calling,came,career,carried,change,child,close,closest,confused,contact,currently,dad,did,didn,disappoint,disturb,divorced,does,doing,don,dreams,easily,end,entire,expectations,expected,experience,extent,extreme,fact,failed,family,fault,feel,feeling,fell,fiasco,fight,finally,firstly,focus,friend,frustrated,getting,gf,girlfriend,going,gone,good,got,grow,guy,hasn,haven,having,help,hidden,hide,hope,hopes,hour,irritated,just,kept,kid,kids,kind,knew,know,knows,life,like,little,live,ll,lost,lot,love,loved,meet,messaged,messages,minutes,mom,months,morning,need,needed,negative,negatives,neverwant,nice,numb,ones,online,overall,parents,people,person,phone,physiatrist,point,prefer,prevent,reason,relations,relationship,relationships,reserved,response,right,seeing,sent,setting,similar,spare,spending,spoiling,spoke,started,stepmom,stepping,sticking,strained,stressful,suggesting,support,talk,tense,therapist,think,thinks,till,time,tips,tldr,today,told,took,tried,trying,understand,upset,used,ve,vent,vomiting,want,way,weird,went,world,worried,year
"I'm feeling very weird and confused right now, a friend I had lost contact with for a few months messaged me today and sent me messages like ""the world has fell away from me"" and ""it's the end for me."" After trying to call me. \n\nI sent him a few confused messaged before I finally knew what he was suggesting and then told him to call someone to talk to them. \n\nCurrently I haven't get any response after an hour and 40 minutes and I've tried calling him too. I'm worried he's done something. I hope he hasn't gone through with it.\n\nI should have answered the call, but it's 3am. I didn't know he needed help.\n\nThis whole experience has made me upset but also a little numb, I don't want to have been able to prevent it but failed. But what if I could have done something?",0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,2,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,1,0,1,0,1,1,0
"Firstly, I love my family very much. I am very blessed to have what we have. \nAs an adult (almost 30) only child I am expected to do almost everything to support our family. I am all my parents hopes and dreams. I have no help, no one to vent to, and a lot of expectations to disappoint. Does anyone have tips for setting up boundaries or stepping up for myself? I would neverwant kids to grow up like I did.",0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
"I think I need more answers, help, and to understand what's going on in my life. So, I kind of want to find a therapist, physiatrist, or something similar. But I have no way of getting one in person, at least.\nAre there any good alternatives anyone knows of? Online or just something similar.",0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
"TLDR: I prefer being alone all the time. Even when I am with my loved ones, I prefer seeing my phone, or spending time by myself. I also get frustrated very easily with the people I’m close to. All I can see are the negatives in them. This is spoiling all my relationships. I know the f

In [11]:
#  Map each 
print("Vocabulary: ", cv.vocabulary_)
vector = cv.transform(df_dtm)

Vocabulary:  {'feeling': 53, 'weird': 168, 'confused': 27, 'right': 129, 'friend': 60, 'lost': 94, 'contact': 28, 'months': 103, 'messaged': 99, 'today': 155, 'sent': 131, 'messages': 100, 'like': 90, 'world': 170, 'fell': 54, 'away': 13, 'end': 41, 'trying': 159, 'finally': 57, 'knew': 86, 'suggesting': 144, 'told': 156, 'talk': 146, 'currently': 29, 'haven': 72, 'response': 128, 'hour': 79, '40': 4, 'minutes': 101, 've': 163, 'tried': 158, 'calling': 19, 'worried': 171, 'hope': 77, 'hasn': 71, 'gone': 66, 'answered': 8, '3am': 3, 'didn': 32, 'know': 87, 'needed': 106, 'help': 74, 'experience': 45, 'upset': 161, 'little': 91, 'numb': 111, 'don': 38, 'want': 166, 'able': 5, 'prevent': 122, 'failed': 49, 'firstly': 58, 'love': 96, 'family': 50, 'blessed': 17, 'adult': 6, '30': 2, 'child': 24, 'expected': 44, 'support': 145, 'parents': 115, 'hopes': 78, 'dreams': 39, 'vent': 164, 'lot': 95, 'expectations': 43, 'disappoint': 33, 'does': 36, 'tips': 153, 'setting': 132, 'boundaries': 18, '

In [12]:
# Sparse Matrix
vector = cv.transform(df_dtm)
vector.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])